
# Sesión 2 — Laboratorio guiado “NumPy mínimo” (antes de la hoja de ejercicios)

Este notebook **no es evaluable**: es un laboratorio para que ganes soltura operativa con NumPy *sin perder el modelo mental*.

**Regla de oro durante todo el notebook:** cuando crees o transformes un array, revisa al menos una vez:
- `shape`
- `dtype`
- `ndim`

Al final serás capaz de:
- crear tus propios datos (reproducibles) con `np.random.default_rng`
- introducir **outliers** de forma controlada
- usar **máscaras booleanas** (selección condicional sin bucles)
- entender `axis` y evitar errores silenciosos
- verificar con visualizaciones mínimas (no decorativas)

---



## 0. Preparación

Ejecuta esta celda para importar librerías y fijar un generador aleatorio reproducible.

- Usaremos `default_rng(seed)` para que tú y tu profesor veáis resultados comparables.
- Usaremos `matplotlib` solo como herramienta de verificación.


In [1]:
import numpy as np
import matplotlib.pyplot as plt

rng = np.random.default_rng(0)  # cambia el 0 si quieres otro escenario reproducible
print("NumPy:", np.__version__)

NumPy: 2.0.2



## 1. Arrays: crear, inspeccionar y **no confiar solo en lo impreso**

NumPy trabaja con `ndarray`, un objeto matemático con contrato estructural:
- valores
- `dtype` (tipo numérico)
- `shape` (forma)
- `ndim` (número de dimensiones)

### 1.1 Crear arrays 1D y 2D


In [2]:
# 1D
a = np.array([1, 2, 3, 4])
print("a =", a)
print("type:", type(a))
print("shape:", a.shape, "dtype:", a.dtype, "ndim:", a.ndim)

# 2D (matriz)
M = np.array([[1, 2, 3],
              [4, 5, 6]])
print("\nM =\n", M)
print("shape:", M.shape, "dtype:", M.dtype, "ndim:", M.ndim)

a = [1 2 3 4]
type: <class 'numpy.ndarray'>
shape: (4,) dtype: int64 ndim: 1

M =
 [[1 2 3]
 [4 5 6]]
shape: (2, 3) dtype: int64 ndim: 2



### 1.2 (n,) vs (1,n) vs (n,1)

En ML, estas tres formas pueden cambiar el significado (y el broadcasting).  
No son “detalles”: son **contratos**.


In [ ]:
x = np.array([10, 20, 30])
x_row = x.reshape(1, 3)   # fila (1,n)
x_col = x.reshape(3, 1)   # columna (n,1)

print("x:", x, "shape:", x.shape)
print("x_row:\n", x_row, "shape:", x_row.shape)
print("x_col:\n", x_col, "shape:", x_col.shape)


## 2. `dtype`: el tipo es semántico (no un detalle)

NumPy fuerza **homogeneidad** de tipo dentro de un array.  
A veces NumPy **promociona** tipos automáticamente (por ejemplo, si aparece un decimal).

### 2.1 Promoción de tipos


In [ ]:
t1 = np.array([21, 22, 23])      # enteros
t2 = np.array([21, 22.5, 23])     # aparece un float -> promoción

print("t1:", t1, "dtype:", t1.dtype)
print("t2:", t2, "dtype:", t2.dtype)

print("mean(t1):", t1.mean())
print("mean(t2):", t2.mean())


### 2.2 Ser explícito con `dtype` cuando el significado importa

Esto no es para “hacerlo más rápido”, sino para evitar sorpresas.


In [ ]:
t_int = np.array([21, 22.5, 23], dtype=np.int32)     # 22.5 -> 22 (truncado)
t_float = np.array([21, 22.5, 23], dtype=np.float64)

print("t_int:", t_int, "dtype:", t_int.dtype, "mean:", t_int.mean())
print("t_float:", t_float, "dtype:", t_float.dtype, "mean:", t_float.mean())


## 3. Operaciones vectorizadas: transformar conjuntos (sin bucles)

Aquí cambia el modelo mental: operas sobre el conjunto completo.

### 3.1 Operaciones básicas


In [ ]:
v = np.array([1, 2, 3, 4, 5], dtype=np.float64)

print("v:", v, "shape:", v.shape, "dtype:", v.dtype)
print("v * 2:", v * 2)
print("v + 10:", v + 10)
print("v ** 2:", v ** 2)


## 4. Máscaras booleanas (selección condicional sin bucles)

Una **máscara** es un array de `True/False` con la misma forma (o compatible) que el dato.

Patrón típico:
1) creas una condición (`mask = x < 0`)
2) usas esa máscara para seleccionar o asignar (`x[mask] = ...`)

### 4.1 Crear una máscara y filtrar


In [ ]:
x = np.array([10, -3, 5, 0, -8, 7], dtype=np.float64)
mask_neg = x < 0

print("x:", x)
print("mask_neg:", mask_neg, "dtype:", mask_neg.dtype)
print("valores negativos:", x[mask_neg])


### 4.2 Asignación con máscara (limpieza mínima)

Ejemplo: sustituir negativos por `np.nan` o por 0.

> Nota: para usar `np.nan` necesitas un array de tipo float.


In [ ]:
y = np.array([50, 45, -3, 60, 0, -10], dtype=np.float64)
mask_invalid = y < 0

y_nan = y.copy()
y_nan[mask_invalid] = np.nan

y_zero = y.copy()
y_zero[mask_invalid] = 0

print("y:", y)
print("y_nan:", y_nan)
print("y_zero:", y_zero)
print("n_invalid:", mask_invalid.sum(), "pct_invalid:", mask_invalid.mean() * 100)


## 5. Broadcasting: potencia… y fuente de errores si no miras `shape`

Regla informal: NumPy alinea por la derecha y expande dimensiones de tamaño 1.

### 5.1 Caso típico correcto: (m,n) + (n,)


In [ ]:
A = np.array([[10, 20, 30],
               [40, 50, 60]], dtype=np.float64)
offset = np.array([1, 2, 3], dtype=np.float64)

print("A.shape:", A.shape, "offset.shape:", offset.shape)
print("A + offset =\n", A + offset)


### 5.2 Caso que debe fallar: (m,n) + (m,)

Si falla, es buena noticia: evita un resultado ambiguo.


In [ ]:
row_offset = np.array([100, 200], dtype=np.float64)
print("A.shape:", A.shape, "row_offset.shape:", row_offset.shape)

try:
    print(A + row_offset)
except Exception as e:
    print("ERROR (esperado):", repr(e))


### 5.3 Hacer explícita la intención con `reshape`

Si quieres sumar/restar un valor por fila, conviértelo en (m,1).


In [ ]:
row_offset = np.array([100, 200], dtype=np.float64).reshape(2, 1)
print("row_offset.shape:", row_offset.shape)
print("A + row_offset =\n", A + row_offset)


## 6. `axis`: el punto donde muchos se pierden

Piensa así:
- `axis=0` agrega “hacia abajo” (colapsa filas → resultado por columnas)
- `axis=1` agrega “hacia la derecha” (colapsa columnas → resultado por filas)

### 6.1 Sumas y medias por eje


In [ ]:
B = np.array([[10, 20, 30],
               [40, 50, 60],
               [70, 80, 90]], dtype=np.float64)

print("B=\n", B)
print("sum total:", B.sum())
print("sum axis=0 (por columnas):", B.sum(axis=0), "shape:", B.sum(axis=0).shape)
print("sum axis=1 (por filas):", B.sum(axis=1), "shape:", B.sum(axis=1).shape)

print("mean axis=0:", B.mean(axis=0))
print("mean axis=1:", B.mean(axis=1))


### 6.2 El error silencioso clásico al centrar por filas

Objetivo: restar a cada fila su media para comparar patrones.

- Versión incorrecta (funciona pero no hace lo que crees)
- Versión correcta con `reshape`


In [ ]:
wrong = B - B.mean(axis=1)  # ¡peligroso!
mu = B.mean(axis=1).reshape(-1, 1)
correct = B - mu

print("B.mean(axis=1):", B.mean(axis=1), "shape:", B.mean(axis=1).shape)
print("mu.shape:", mu.shape)

print("\nMedia por fila (wrong):", wrong.mean(axis=1))
print("Media por fila (correct):", correct.mean(axis=1))


## 7. Crear tus propios datos (reales, reproducibles) + introducir outliers

Esto es importante: en IA no siempre te dan un dataset “perfecto”.  
Aquí practicas cómo **generar** datos para testear ideas y cómo **simular** problemas reales (outliers).

### 7.1 Serie temporal simple (7 días × 24 horas = 168 puntos)
- base estable + ruido
- 5 picos anómalos (outliers)


In [ ]:
# Base + ruido
base = 1.5
noise = rng.normal(0, 0.25, size=168)
consumo = base + noise
consumo = np.clip(consumo, 0, None).astype(np.float64)

# Insertar outliers
idx_outliers = rng.choice(np.arange(consumo.size), size=5, replace=False)
consumo_out = consumo.copy()
consumo_out[idx_outliers] += rng.uniform(5, 10, size=5)

print("shape:", consumo_out.shape, "dtype:", consumo_out.dtype)
print("min/max/mean/median:", consumo_out.min(), consumo_out.max(), consumo_out.mean(), np.median(consumo_out))
print("idx_outliers:", np.sort(idx_outliers))


### 7.2 Verificación rápida con gráficos (no decorativos)

- histograma: ¿cola larga? ¿outliers dominan?
- línea temporal: ¿dónde están los picos?


In [ ]:
plt.figure()
plt.hist(consumo_out, bins=50)
plt.title("Histograma consumo (raw)")
plt.show()

plt.figure()
plt.plot(consumo_out)
plt.title("Serie temporal consumo (raw)")
plt.show()


### 7.3 `log1p`: ver mejor cuando hay cola larga

`log1p(x)` aplica log(1+x), es estable con ceros y reduce el impacto visual de valores grandes.


In [ ]:
plt.figure()
plt.hist(np.log1p(consumo_out), bins=50)
plt.title("Histograma consumo (log1p)")
plt.show()


## 8. Mini-caso: agregación diaria con `reshape` + `axis`

Queremos pasar de 168 puntos (7×24) a 7 valores diarios:
- total diario
- media diaria

**Ojo:** si haces mal el `reshape` o el `axis`, el código “funciona”, pero el significado cambia.


In [ ]:
C = consumo_out.reshape(7, 24)   # día x hora
total_dia = C.sum(axis=1)
media_dia = C.mean(axis=1)

print("C.shape:", C.shape)
print("total_dia:", total_dia, "shape:", total_dia.shape)
print("media_dia:", media_dia, "shape:", media_dia.shape)

print("día max total:", int(total_dia.argmax()), "día max media:", int(media_dia.argmax()))


## 9. Cierre: checklist de hábitos que se evaluarán a partir de ahora

Antes de modelar (y durante todo el curso), deberías poder hacer sin esfuerzo:

1) Inspeccionar: `shape`, `dtype`, `ndim`  
2) Transformar: operaciones vectorizadas (sin bucles)  
3) Seleccionar/limpiar: **máscaras booleanas**  
4) Operar con intención: `axis` y `reshape` cuando haga falta  
5) Verificar: un gráfico que responda una pregunta (outliers, señal, escala)

Cuando termines, si quieres practicar:
- cambia la semilla del RNG
- cambia el número de outliers
- cambia el patrón (picos concentrados en un día, o repartidos)
- repite la agregación diaria y compara historias (total vs media)
